**DATASET2**

fluid_vs_infomap =0.7


leiden_vs_infomap =0.0

**DATASET 3**


fluid_vs_infomap =0.55

leiden_vs_infomap =0.0

**DATASET 4**

fluid_vs_infomap =0.0

leiden_vs_infomap =0.0

In [ ]:
pip install leidenalg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 23.8 MB/s 


In [ ]:
pip install python-louvain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install infomap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 262 kB 5.0 MB/s 
  Created wheel for infomap: filename=infomap-2.6.1-cp38-cp38-linux_x86_64.whl size=5635253 sha256=be00c9c5e20ab91a8da2e8310d640afbc6ee92fc595769e9e405b2e165f42013
  Stored in directory: /root/.cache/pip/wheels/18/df/51/25cf260bb989de3e0819c7ddd672b989274c730c550dbf545b
Successfully built infomap


In [ ]:
pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install community

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# This code is an extention of the cada code available at
# https://github.com/thomashelling/cada
#
# coding=utf-8

import networkx as nx
from networkx.algorithms.community import asyn_fluidc
from networkx.algorithms.community import greedy_modularity_communities as gmc
import random
import numpy as np
import community
import time
import infomap
import igraph as ig
import leidenalg

class cada():
	def __init__(self, graph, algorithm='louvain', resolution=0.1):
		
		# First do community detection
		if algorithm == 'louvain':
			partition = community.best_partition(graph, resolution=resolution)
		elif algorithm == 'leiden':
			partition = self.run_leiden(graph,resolution=resolution)
		elif algorithm == 'fluid':
			partition = self.run_fluid(graph)
		elif algorithm == 'greedy':
			partition = self.run_greedy(graph,resolution)
		else:
			partition = self.run_infomap(graph)
		
		communities = set()
		for node in graph.nodes():
			if node in partition:
				communities.add(partition[node])

		anom_score = {}

		# Compute anomaly score for each node
		for node in graph.nodes():
			comms = {}
			for neighbor in graph.neighbors(node):
				if neighbor != node:
					if partition[neighbor] not in comms:
						comms[partition[neighbor]] = 0

					comms[partition[neighbor]] += 1

			if len(comms) > 0:
				# The number of communities it is connected to. 
				comms = np.array(list(comms.values()))
				# print('nr communities connected', comms)
				max_com = np.max(comms)
				# print('Maxcommunity', max_com)
				comms = comms / max_com
				# print('Communities normalized', comms)
				anom_score[node] = np.sum(comms)		
				# print('Anomaly score., ', anom_score[node])

		self.anomaly_scores = sorted(anom_score.items(), key=lambda x: x[1])[::-1]
		self.n_nodes = graph.number_of_nodes()

	def clf(self,n_injections):
		"""
		Predicts n_injections anomaly nodes 
		"""
		pred = np.zeros(self.n_nodes,dtype=int)
		#worst_nodes = self.anomaly_scores[:n_injections][0]
		for ind,anomaly in enumerate(self.anomaly_scores):
			if(ind>=n_injections):
				break
			pred[anomaly[0]] = 1
			
		return pred

	def run_leiden(self,graph,resolution):
		"""
		Creates partitions with the Leiden algorithm
		"""
		h = ig.Graph.from_networkx(graph)
		#part = leidenalg.find_partition(h, leidenalg.ModularityVertexPartition)
		#part = leidenalg.find_partition(h, leidenalg.RBConfigurationVertexPartition,resolution_parameter = 0.5)
		part = leidenalg.find_partition(h, leidenalg.CPMVertexPartition,resolution_parameter = resolution)
		partition = {}

		for ind,com in enumerate(part):
			for node in com:
				partition[node] = ind

		return partition

	def run_greedy(self,graph,resolution):
		"""
		Creates partitions with Clauset-Newman-Moore greedy modularity maximization
		"""
		best_n = int(graph.number_of_nodes()/10)
		communities = gmc(graph,resolution=resolution,best_n=best_n)
		partition = {}
		print(f"nr of coms={len(communities)}")
		for ind,com in enumerate(communities):
			for node in com:
				partition[node] = ind
		
		return partition

	def run_fluid(self,graph):
		"""
		Creates partitions with async_fluidc
		"""
		k = int(graph.number_of_nodes()/100)
		it = asyn_fluidc(graph,k=k)
		partition = {}

		for ind,com in enumerate(it):
			for node in com:
				partition[node] = ind

		return partition

	def run_infomap(self, graph):
		"""
		Runs Infomap with infomap package 
		"""
		infomapSimple = infomap.Infomap("--two-level --silent")
		network = infomapSimple.network
		
		for e in graph.edges():
			network.addLink(e[0], e[1])

		partition = {}
		infomapSimple.run()
		for node in infomapSimple.iterTree():
			if node.isLeaf():
				partition[node.physicalId] = node.moduleIndex()

		return partition

	def get_anomaly_scores(self, nr_anomalies=None):
		"""
		Returns tuple (node, anomaly_score) for either nr_anomalies or all
		"""
		if nr_anomalies:
			return self.anomaly_scores[:nr_anomalies]
		else:
			return self.anomaly_scores 

	def get_top_anomalies(self, nr_anomalies=100):
		"""
		Returns highest scoring anomalies
		"""					
		anomalies = []
		for anomaly in self.anomaly_scores[:nr_anomalies]:
			anomalies.append(anomaly[0])

		return anomalies

	def get_anomalies_threshold(self, threshold):
		"""
		Returns anomalies that are above a certain threshold.
		"""
		anomalies = []

		for anomaly in self.anomaly_scores:
			if anomaly[1] > threshold:
				anomalies.append(anomaly[0])
			else:
				break

		return anomalies

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from time import process_time as pt
import pandas as pd

**DATASET 1**

In [ ]:
amazons.dropna(inplace=True)
amazons.drop([3],axis=0,inplace=True)
amazons.reset_index(inplace=True,drop=True)

In [ ]:
amazons

,Source,Destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
3387383,403392,121379
3387384,403392,190663
3387385,403393,318438
3387386,403393,326962


In [ ]:
amazons = amazons.astype({'Source':'int'})
amazons = amazons.astype({'Destination':'int'})

In [ ]:
amazons.iloc[0]

Source         0
Destination    1
Name: 0, dtype: int64

In [ ]:
amazons[amazons['Source'] == 403376]

,Source,Destination
3387243,403376,45
3387244,403376,1039
3387245,403376,1041
3387246,403376,22120
3387247,403376,113347
3387248,403376,121379
3387249,403376,403270
3387250,403376,403375
3387251,403376,403391


In [ ]:
amazon = nx.from_pandas_edgelist(amazons, source='Source', target='Destination',create_using=nx.DiGraph)
amazon = amazon.to_undirected()
#amazon_in_degrees = sorted((d for n, d in amazon.in_degree()), reverse=True)
#amazon_out_degrees = sorted((d for n, d in amazon.out_degree()), reverse=True)

#largest_cc =nx.subgraph(amazon,max(nx.weakly_connected_components(amazon),key=len))
temp = nx.from_pandas_edgelist(amazons, source='Source', target='Destination',create_using=nx.DiGraph)
largest_cc =nx.subgraph(temp,max(nx.weakly_connected_components(temp),key=len))
largest_cc = largest_cc.to_undirected()

In [ ]:
#list(set(amazon.nodes) - set(largest_cc.nodes))

[]

In [ ]:
#for i in list(set(amazon.nodes) - set(largest_cc.nodes)):
  #amazon.remove_node(i)

In [ ]:
### LEIDEN
cada_leiden = cada(amazon,algorithm='leiden')
n_inj = int(100)
pred_leiden = cada_leiden.clf(n_inj)

In [ ]:
### INFOMAP
cada_infomap = cada(amazon,algorithm='infomap')
n_inj = int(100)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
### Fluid Communities require connected Graphs.

cada_fluid = cada(largest_cc,algorithm='fluid')
pred_fluid= cada_fluid.clf(100)

In [ ]:
pred_fluid.shape

(403364,)

In [ ]:
print('fluid_vs_infomap =' + str(np.sum(pred_fluid * pred_infomap) / 100))

fluid_vs_infomap =0.15


In [ ]:
print('leiden_vs_infomap =' + str(np.sum(pred_leiden * pred_infomap) / 100))

leiden_vs_infomap =0.15


In [ ]:
### module 'community' has no attribute 'best_partition'
#cada_lo = cada(amazon,algorithm='louvain')
#pred_lo = cada_lo.clf(n_inj)

**DATASET 2**

In [ ]:
emails

,Source,Destination
0,# Directed graph (each unordered pair of nodes...,NaN
1,# Email network of a large European Research I...,NaN
2,# Nodes: 265214 Edges: 420045,NaN
3,# FromNodeId,ToNodeId
4,0,1
...,...,...
420044,265209,422
420045,265210,422
420046,265211,192
420047,265212,875


In [ ]:
emails.dropna(inplace=True)
emails.drop([3],axis=0,inplace=True)
emails.reset_index(inplace=True,drop=True)

In [ ]:
emails = emails.astype({'Source':'int'})
emails = emails.astype({'Destination':'int'})

In [ ]:
email = nx.from_pandas_edgelist(emails, source='Source', target='Destination',create_using=nx.DiGraph)
email = email.to_undirected()
#email_in_degrees = sorted((d for n, d in email.in_degree()), reverse=True)
#email_out_degrees = sorted((d for n, d in email.out_degree()), reverse=True)
#temp = nx.from_pandas_edgelist(emails, source='Source', target='Destination',create_using=nx.DiGraph)
#largest_cc =nx.subgraph(temp,max(nx.weakly_connected_components(temp),key=len))
#largest_cc = largest_cc.to_undirected()

In [ ]:
louvain1 = louvain1.T.reset_index().T.reset_index(drop=True)

In [ ]:
louvain1[0][0] = 0
louvain1[0]

0           0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
265209    0.0
265210    0.0
265211    0.0
265212    0.0
265213    0.0
Name: 0, Length: 265214, dtype: object

In [ ]:
np.array(louvain1).shape

(265214, 1)

In [ ]:
pred_louvain = np.array(louvain1).reshape(265214,)

In [ ]:
### LEIDEN
cada_leiden = cada(email,algorithm='leiden')
n_inj = int(10)
pred_leiden = cada_leiden.clf(n_inj)

In [ ]:
pred_leiden.shape

(265214,)

In [ ]:
### INFOMAP
cada_infomap = cada(email,algorithm='infomap')
n_inj = int(10)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
print('leiden_vs_infomap =' + str(np.sum(pred_leiden * pred_infomap) / n_inj))
print('leiden_vs_louvain =' + str(np.sum(pred_leiden * pred_louvain) / n_inj))
print('louvain_vs_infomap =' + str(np.sum(pred_louvain * pred_infomap) / n_inj))


leiden_vs_infomap =0.0
leiden_vs_louvain =0.0
louvain_vs_infomap =0.8


In [ ]:
#from numpy import savetxt
#savetxt('data.csv', pred_leiden, delimiter=',')
# load numpy array from csv file
#from numpy import loadtxt
# load array
#data = loadtxt('data.csv', delimiter=',')

In [ ]:
### Fluid Communities require connected Graphs.

cada_fluid = cada(largest_cc,algorithm='fluid')
pred_fluid= cada_fluid.clf(10)

In [ ]:
print('fluid_vs_infomap =' + str(np.sum(pred_fluid * pred_infomap) / 10))

fluid_vs_infomap =0.7


**DATASET2**

fluid_vs_infomap =0.7


leiden_vs_infomap =0.0

**DATASET 3**


fluid_vs_infomap =0.55

leiden_vs_infomap =0.0




In [ ]:
wiki

,Source,Destination
0,# Directed graph (each unordered pair of nodes...,NaN
1,# Communication network of Wikipedia (till Jan...,NaN
2,# Nodes: 2394385 Edges: 5021410,NaN
3,# FromNodeId,ToNodeId
4,0,1
...,...,...
5021409,2393724,2393725
5021410,2393740,80239
5021411,2393816,27171
5021412,2393817,27171


In [ ]:
wiki.dropna(inplace=True)
wiki.drop([3],axis=0,inplace=True)
wiki.reset_index(inplace=True,drop=True)

In [ ]:
wiki = wiki.astype({'Source':'int'})
wiki = wiki.astype({'Destination':'int'})

In [ ]:
wiki_g = nx.from_pandas_edgelist(wiki, source='Source', target='Destination',create_using=nx.DiGraph)
wiki_g = wiki_g.to_undirected()
#email_in_degrees = sorted((d for n, d in email.in_degree()), reverse=True)
#email_out_degrees = sorted((d for n, d in email.out_degree()), reverse=True)

In [ ]:
louvain2 = louvain2.T.reset_index().T.reset_index(drop=True)
louvain2

,0
0,0.000000000000000000e+00
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2394380,0.0
2394381,0.0
2394382,0.0
2394383,0.0


In [ ]:
louvain2[0][0] = 0

In [ ]:
louvain2.sum()

0    100.0
dtype: object

In [ ]:
np.array(louvain2).shape

(2394385, 1)

In [ ]:
pred_louvain = np.array(louvain2).reshape(2394385,)

In [ ]:
### LEIDEN
cada_leiden = cada(wiki_g,algorithm='leiden')
n_inj = int(100)
pred_leiden = cada_leiden.clf(n_inj)

In [ ]:
### INFOMAP
cada_infomap = cada(wiki_g,algorithm='infomap')
n_inj = int(100)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
print('leiden_vs_infomap =' + str(np.sum(pred_leiden * pred_infomap) / n_inj))
print('leiden_vs_louvain =' + str(np.sum(pred_leiden * pred_louvain) / n_inj))
print('louvain_vs_infomap =' + str(np.sum(pred_louvain * pred_infomap) / n_inj))

leiden_vs_infomap =0.0
leiden_vs_louvain =0.0
louvain_vs_infomap =0.34


In [ ]:
temp = nx.from_pandas_edgelist(wiki, source='Source', target='Destination',create_using=nx.DiGraph)
largest_cc =nx.subgraph(temp,max(nx.weakly_connected_components(temp),key=len))
largest_cc = largest_cc.to_undirected()

In [ ]:
### INFOMAP
cada_infomap = cada(largest_cc,algorithm='infomap')
n_inj = int(100)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
### Fluid Communities require connected Graphs.

cada_fluid = cada(largest_cc,algorithm='fluid')
pred_fluid= cada_fluid.clf(100)

In [ ]:
print('fluid_vs_infomap =' + str(np.sum(pred_fluid * pred_infomap) / 100))

fluid_vs_infomap =0.55


**DATASET 4**

fluid_vs_infomap =0.0

leiden_vs_infomap =0.0

In [ ]:
louvain3 = louvain3.T.reset_index().T.reset_index(drop=True)
louvain3

,0
0,0.000000000000000000e+00
1,0.0
2,0.0
3,0.0
4,0.0
...,...
168109,0.0
168110,0.0
168111,0.0
168112,0.0


In [ ]:
louvain3[0][0] = 0

In [ ]:
louvain3.sum()

0    100.0
dtype: object

In [ ]:
np.array(louvain3).shape

(168114, 1)

In [ ]:
pred_louvain = np.array(louvain3).reshape(168114,)

In [ ]:
twitch.drop([0],axis=0,inplace=True)
twitch.reset_index(inplace=True,drop=True)

In [ ]:
twitch


,Source,Destination
0,98343,141493
1,98343,58736
2,98343,140703
3,98343,151401
4,98343,157118
...,...,...
6797552,97507,29359
6797553,71175,12020
6797554,151702,128281
6797555,118034,38021


In [ ]:
twitch = twitch.astype({'Source':'int'})
twitch = twitch.astype({'Destination':'int'})

In [ ]:
twitch_g = nx.from_pandas_edgelist(twitch, source='Source', target='Destination',create_using=nx.DiGraph)
twitch_g = twitch_g.to_undirected()

#email_in_degrees = sorted((d for n, d in email.in_degree()), reverse=True)
#email_out_degrees = sorted((d for n, d in email.out_degree()), reverse=True)

In [ ]:
#print(nx.average_clustering(wiki_g))

In [ ]:
### LEIDEN
cada_leiden = cada(twitch_g,algorithm='leiden')
n_inj = int(100)
pred_leiden = cada_leiden.clf(n_inj)

In [ ]:
### INFOMAP
cada_infomap = cada(twitch_g,algorithm='infomap')
n_inj = int(100)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
print('leiden_vs_infomap =' + str(np.sum(pred_leiden * pred_infomap) / n_inj))
print('leiden_vs_louvain =' + str(np.sum(pred_leiden * pred_louvain) / n_inj))
print('louvain_vs_infomap =' + str(np.sum(pred_louvain * pred_infomap) / n_inj))

leiden_vs_infomap =0.0
leiden_vs_louvain =0.0
louvain_vs_infomap =0.01


In [ ]:
print('leiden_vs_infomap =' + str(np.sum(pred_leiden * pred_infomap) / n_inj))

leiden_vs_infomap =0.0


In [ ]:
temp = nx.from_pandas_edgelist(twitch, source='Source', target='Destination',create_using=nx.DiGraph)
largest_cc =nx.subgraph(temp,max(nx.weakly_connected_components(temp),key=len))
largest_cc = largest_cc.to_undirected()

In [ ]:
### INFOMAP
cada_infomap = cada(largest_cc,algorithm='infomap')
n_inj = int(100)
pred_infomap = cada_infomap.clf(n_inj)

In [ ]:
### Fluid Communities require connected Graphs.

cada_fluid = cada(largest_cc,algorithm='fluid')
pred_fluid= cada_fluid.clf(100)

In [ ]:
print('fluid_vs_infomap =' + str(np.sum(pred_fluid * pred_infomap) / 100))

fluid_vs_infomap =0.0
